In [1]:
import requests
from bs4 import BeautifulSoup
import re
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import time
import csv
import os
import pickle
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import graphviz
from sklearn.neighbors import KNeighborsRegressor

!pip install keras==2.12.0

import tensorflow as tf

In [2]:
# !pip uninstall tensorflow
# !pip install tensorflow==2.12.0

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor # There is also a KerasClassifier class
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

!pip install openpyxl
import openpyxl
print("Library versions: pandas", pd.__version__," numpy", np.__version__," seaborn", sns.__version__)

Library versions: pandas 2.2.2  numpy 1.23.5  seaborn 0.13.2


In [14]:
def download_data():
    URL = 'https://fossilfreefunds.org/how-it-works'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    urls = []
    names = []
    for i, link in enumerate(soup.findAll('a')):
        FULLURL = link.get('href')

        if bool(re.search('.*results.*.xlsx', str(FULLURL))):
            urls.append(FULLURL)
            names.append(os.path.basename(soup.select('a')[i].attrs['href']))

    names_urls = zip(names, urls)
    for name, url in names_urls:
        print("Download file: "+name)
        r = requests.get(url, verify=False,stream=True)
        r.raw.decode_content = True
        with open("/content/data" + name, 'wb') as out:
                shutil.copyfileobj(r.raw, out)

def merge_excel():
    df = pd.DataFrame()
    files=os.listdir('data')
    files_xls = [f for f in files if f[-4:]=='xlsx']

    for f in files_xls:
        if not re.match(r".*20210[5-9]+.*", f):
            print('Merging file: '+f)
            data = pd.read_excel('data/'+f, 'Shareclasses',engine='openpyxl')
            df = df.append(data)
    df.to_csv('/content/data/fossilfund_dataset.csv', index=False)
    print('Export to data/fossilfund_dataset.csv is finished')

download_data()
merge_excel()

Export to data/fossilfund_dataset.csv is finished


In [15]:
df = pd.read_csv('/content/data/fossilfund_dataset.csv')

<ipython-input-15-c21305b81df0>:1: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/data/fossilfund_dataset.csv')


In [16]:
print("Total no. of columns in the dataframe", len(df.columns))
print("No. of columns containing null values", len(df.columns[df.isna().any()]))

print("No. of columns not containing null values", len(df.columns[df.notna().all()]))
print("No. of numerical columns ", len(df.select_dtypes(np.number).columns))

print("Total no. of rows in the dataframe", len(df))

Total no. of columns in the dataframe 121
No. of columns containing null values 42
No. of columns not containing null values 79
No. of numerical columns  101
Total no. of rows in the dataframe 110510


In [17]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110510 entries, 0 to 110509
Data columns (total 121 columns):
 #    Column                                                                                                          Non-Null Count   Dtype  
---   ------                                                                                                          --------------   -----  
 0    Fund profile: Shareclass name                                                                                   110510 non-null  object 
 1    Fund profile: Ticker                                                                                            108086 non-null  object 
 2    Fund profile: Fund name                                                                                         110510 non-null  object 
 3    Fund profile: Asset manager                                                                                     110510 non-null  object 
 4    Fund profile: Shareclass

### Converting date-time

In [18]:
date_cols=df.filter(regex=" date.*",axis=1).columns
df[date_cols]=df[date_cols].apply(pd.to_datetime, errors='coerce')
df[date_cols]

,Fund profile: Shareclass inception date,Fund profile: Oldest shareclass inception date,Fund profile: Portfolio holdings as-of date,Financial performance: Financial performance as-of date
0,2014-11-12,2014-11-12,2020-01-31,2020-03-31
1,2014-11-12,2014-11-12,2020-01-31,2020-03-31
2,2014-11-12,2014-11-12,2020-01-31,2020-03-31
3,2014-11-12,2014-11-12,2020-01-31,2020-03-31
4,2011-12-28,2011-12-28,2019-12-31,2020-03-31
...,...,...,...,...
110505,2018-12-04,2018-12-04,2021-02-28,2021-02-28
110506,2018-09-05,2018-09-05,2021-02-28,2021-02-28
110507,2018-12-04,2018-12-04,2021-02-28,2021-02-28
110508,2019-03-06,2019-03-06,2021-02-28,2021-02-28


In [19]:
# These are the numerical features per category

origin_categories=["Fund profile","Fossil Free Funds", "Deforestation Free Funds", "Gender Equality Funds", "Gun Free Funds", "Prison Free Funds", "Weapon Free Funds", "Tobacco Free Funds", "Financial performance"]
for category in origin_categories:
    print("Distribution of numerical features for category: "+category)
    display(HTML(df.filter(regex=category+".*",axis=1).describe().to_html()))
    print("\n\n")

Distribution of numerical features for category: Fund profile


,Fund profile: Shareclass inception date,Fund profile: Oldest shareclass inception date,Fund profile: Portfolio holdings as-of date,Fund profile: Fund net assets,Fund profile: Percent rated
count,110510,110510,110510,1.105100e+05,110510.000000
mean,2007-10-24 15:59:48.011944448,2000-08-19 19:46:11.521129344,2020-07-22 18:14:40.731155456,7.421210e+09,0.943398
min,1924-07-15 00:00:00,1924-07-15 00:00:00,2019-02-28 00:00:00,1.534680e+05,0.400349
25%,2002-02-20 00:00:00,1994-05-02 00:00:00,2020-04-30 00:00:00,2.025332e+08,0.951903
50%,2008-11-03 00:00:00,2001-06-01 00:00:00,2020-06-30 00:00:00,8.267780e+08,0.977816
75%,2015-07-01 00:00:00,2011-02-01 00:00:00,2020-09-30 00:00:00,3.094538e+09,0.991256
max,2021-03-01 00:00:00,2021-02-24 00:00:00,2021-02-28 00:00:00,1.075580e+12,1.000000
std,NaN,NaN,NaN,3.493070e+10,0.105523





Distribution of numerical features for category: Fossil Free Funds


,"Fossil Free Funds: Fossil fuel holdings, count","Fossil Free Funds: Fossil fuel holdings, weight","Fossil Free Funds: Fossil fuel holdings, asset","Fossil Free Funds: Carbon Underground 200, count","Fossil Free Funds: Carbon Underground 200, weight","Fossil Free Funds: Carbon Underground 200, asset","Fossil Free Funds: Coal industry, count","Fossil Free Funds: Coal industry, weight","Fossil Free Funds: Coal industry, asset","Fossil Free Funds: Oil / gas industry, count","Fossil Free Funds: Oil / gas industry, weight","Fossil Free Funds: Oil / gas industry, asset","Fossil Free Funds: Macroclimate 30 coal-fired utilities, count","Fossil Free Funds: Macroclimate 30 coal-fired utilities, weight","Fossil Free Funds: Macroclimate 30 coal-fired utilities, asset","Fossil Free Funds: Fossil-fired utilities, count","Fossil Free Funds: Fossil-fired utilities, weight","Fossil Free Funds: Fossil-fired utilities, asset",Fossil Free Funds: Relative carbon footprint (tonnes CO2 / $1M USD invested),Fossil Free Funds: Relative carbon intensity (tonnes CO2 / $1M USD revenue),Fossil Free Funds: Total financed emissions scope 1 + 2 (tCO2e),Fossil Free Funds: Total financed emissions scope 1 + 2 + 3 (tCO2e),Fossil Free Funds: Carbon footprint portfolio coverage by market value weight,Fossil Free Funds: Carbon footprint portfolio coverage by number of disclosing titles,"Fossil Free Funds: Clean200, count","Fossil Free Funds: Clean200, weight","Fossil Free Funds: Clean200, asset"
count,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05,1.105100e+05,110510.000000,110510.000000,110510.000000,110510.000000,1.105100e+05
mean,16.776328,0.066618,4.559884e+08,5.080291,0.022519,1.625754e+08,2.051154,0.005956,3.603254e+07,8.966329,0.035334,2.135207e+08,1.287892,0.007629,8.170273e+07,7.065207,0.030147,2.397661e+08,120.149909,167.592131,8.104975e+05,2.531491e+06,0.902381,0.538557,9.926513,0.074519,6.763798e+08
std,36.225283,0.120382,2.416856e+09,10.294005,0.055667,8.644581e+08,7.080570,0.014675,2.952930e+08,19.619299,0.097442,1.100159e+09,2.575200,0.018861,6.400949e+08,16.051047,0.063305,1.387114e+09,139.569661,194.430358,4.049854e+06,1.285494e+07,0.114800,0.260548,16.027081,0.065907,3.597203e+09
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,1.000000,0.004361,4.081950e+05,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,27.830000,64.202500,1.241923e+04,4.403967e+04,0.890223,0.360000,2.000000,0.017976,2.929204e+06
50%,6.000000,0.042483,1.946916e+07,2.000000,0.009438,2.591610e+06,0.000000,0.000000,0.000000e+00,3.000000,0.017249,7.172282e+06,0.000000,0.000000,0.000000e+00,2.000000,0.018291,5.568960e+06,81.785000,126.530000,6.703449e+04,2.268255e+05,0.939559,0.574074,5.000000,0.067256,3.527957e+07
75%,17.000000,0.081263,1.132204e+08,5.000000,0.027531,3.265208e+07,1.000000,0.005303,3.014772e+06,8.000000,0.038630,5.082529e+07,1.000000,0.008828,6.446923e+06,7.000000,0.040002,4.873230e+07,160.680000,206.030000,2.557974e+05,8.391097e+05,0.967733,0.756972,11.000000,0.109732,2.058332e+08
max,607.000000,1.000000,6.561243e+10,154.000000,0.987275,2.145536e+10,126.000000,0.430865,9.675759e+09,304.000000,1.000000,2.751784e+10,31.000000,0.318989,2.000770e+10,286.000000,0.970667,3.884303e+10,1911.540000,3279.730000,8.837339e+07,3.252584e+08,0.999999,1.000000,210.000000,0.563840,1.246118e+11





Distribution of numerical features for category: Deforestation Free Funds


...

,"Weapon Free Funds: Military weapon, count","Weapon Free Funds: Military weapon, weight","Weapon Free Funds: Military weapon, asset","Weapon Free Funds: Major military contractors, count","Weapon Free Funds: Major military contractors, weight","Weapon Free Funds: Major military contractors, asset","Weapon Free Funds: Nuclear weapons, count","Weapon Free Funds: Nuclear weapons, weight","Weapon Free Funds: Nuclear weapons, asset","Weapon Free Funds: Cluster munitions / landmines, count","Weapon Free Funds: Cluster munitions / landmines, weight","Weapon Free Funds: Cluster munitions / landmines, asset"
count,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05
mean,3.294833,0.016616,1.823725e+08,3.253163,0.016549,1.821343e+08,1.602000,0.009491,1.296912e+08,0.037743,0.000015,6.391814e+04
std,5.221361,0.025743,9.427030e+08,5.147298,0.025762,9.424386e+08,2.556526,0.018837,7.131973e+08,0.365929,0.000218,1.270657e+06
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
25%,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
50%,1.000000,0.011855,5.140992e+06,1.000000,0.011795,4.903307e+06,1.000000,0.000927,7.446500e+04,0.000000,0.000000,0.000000e+00
75%,4.000000,0.025795,4.442682e+07,4.000000,0.025757,4.355494e+07,2.000000,0.015139,1.719410e+07,0.000000,0.000000,0.000000e+00
max,66.000000,0.860331,2.723234e+10,63.000000,0.860331,2.723234e+10,23.000000,0.687890,1.928129e+10,7.000000,0.009169,5.162492e+07





Distribution of numerical features for category: Tobacco Free Funds


,"Tobacco Free Funds: Tobacco producer, count","Tobacco Free Funds: Tobacco producer, weight","Tobacco Free Funds: Tobacco producer, asset","Tobacco Free Funds: Tobacco-promoting entertainment company, count","Tobacco Free Funds: Tobacco-promoting entertainment company, weight","Tobacco Free Funds: Tobacco-promoting entertainment company, asset"
count,110510.000000,110510.000000,1.105100e+05,110510.000000,110510.000000,1.105100e+05
mean,0.792019,0.005017,7.821135e+07,0.710325,0.007367,8.063425e+07
std,1.497490,0.011223,4.810194e+08,1.207540,0.014824,6.045201e+08
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
25%,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
50%,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00
75%,1.000000,0.005041,2.672713e+06,1.000000,0.010001,6.559704e+06
max,19.000000,0.140505,5.913728e+09,8.000000,0.224224,2.237722e+10





Distribution of numerical features for category: Financial performance


,Financial performance: Financial performance as-of date,"Financial performance: Month end trailing returns, year 1","Financial performance: Month end trailing returns, year 3","Financial performance: Month end trailing returns, year 5","Financial performance: Month end trailing returns, year 10"
count,110504,108367.000000,103448.000000,96542.000000,81748.000000
mean,2020-09-06 09:04:37.564613120,8.354840,6.455049,8.315809,9.381584
min,2020-03-31 00:00:00,-79.657770,-50.617880,-39.775530,-25.167160
25%,2020-05-31 00:00:00,-5.553780,0.870672,3.775177,6.437010
50%,2020-08-31 00:00:00,5.761320,5.415205,7.775750,9.396265
75%,2020-10-31 00:00:00,19.653715,11.350495,12.442040,12.601810
max,2021-02-28 00:00:00,237.930500,71.112440,54.301480,24.884430
std,NaN,20.709696,8.775643,7.034999,4.877457


Now we need to check for data completeness and more